In [1]:
import DiFfRG.phasediagram as pd
import DiFfRG.file_io as io
import DiFfRG.plot as plt
import numpy as np

In [224]:
import glob
import os

def load_data():
  global sims, csvs, params
  sims = glob.glob('./PhaseDiagram/*.log')
  sims = [sim[:-4].split('/')[-1] for sim in sims]
  csvs = [f'./PhaseDiagram/{sim}_data.csv' for sim in sims]
  params = [io.get_parameters_from_name(sim) for sim in sims]
  
def clear_data():
  load_data()
  for i, csv in enumerate(csvs):
    if not os.path.exists(csv):
      baseName = './PhaseDiagram/' + sims[i]
      print('removing', baseName)
      os.system(f'rm -r {baseName}/ {baseName}.log {baseName}.log.json')
  load_data()
  
def tell_learner(learner):
  clear_data()
  for i, csv in enumerate(csvs):
    sigma = io.read_csv(csv)["sigmaGeV"].to_numpy()[-1]
    prm = params[i]
    print(i, " ", csv, " : ", sigma, " ", prm)
    learner.tell((prm['m2'], prm['T']), sigma)
  
def get_data():
  load_data()
  global in_data
  in_data = {}
  T = []
  m2 = []
  sigma = []
  for i, csv in enumerate(csvs):
    csv_data = io.read_csv(csv)
    if(csv_data["t"].to_numpy()[-1] < 3.8):
      continue
    msigma = csv_data["sigmaGeV"].to_numpy()[-1]
    prm = params[i]
    T.append(prm['T'])
    m2.append(prm['m2'])
    sigma.append(msigma)
  in_data['T'] = np.array(T)
  in_data['m2'] = np.array(m2)
  in_data['sigma'] = np.array(sigma)

In [ ]:
import adaptive
adaptive.notebook_extension()
from concurrent.futures import ProcessPoolExecutor

folder = "./PhaseDiagram/"
parameter = "./parameter.json"
executable = "./build/CG"
observable_to_learn = "sigmaGeV"
loss_goal = 0.001

specification = [
    {"parameter": "/physical/m2", "bounds": [-0.2, -0.1999]},
    {"parameter": "/physical/T", "bounds": [0.05, 0.25]},
]

def solve_point(point):
    pd_point = [[spec["parameter"], point[i]] for i, spec in enumerate(specification)]
    name = pd.run_point(executable, pd_point, folder=folder, add_params=f"")
    csv = folder + name + "_data.csv" 
    ob = io.read_csv(csv)[observable_to_learn].to_numpy()[-1]
    return ob

bounds = []
for spec in specification:
    bounds.append(spec["bounds"])

learner = adaptive.LearnerND(solve_point, bounds=bounds)
tell_learner(learner)

executor = ProcessPoolExecutor(max_workers=4)
runner = adaptive.Runner(
    learner,
    executor=executor,
    loss_goal=loss_goal,
)
runner.live_info()  # shows a widget with status information
#runner.live_plot()

In [258]:
import importlib
importlib.reload(plt)

get_data()

plt.plot_2D(
    {
        "x" : in_data['m2'] * 1e3,
        "y" : in_data['T'] * 1e3,
        "z" : in_data['sigma'] * 1e3,
    },
    cmap="rocket_r",
    style="3D",
    levels=100,
    grid=True,
    xlabel=r"$m^2\,\textrm{[MeV]}$",
    ylabel=r"$T\,\textrm{[MeV]}$",
    zlabel=r"$\sigma\,\textrm{[MeV]}$",
    xlim = (-200, -100),
    ylim = (50, 230),
    zlim = (0, 90),
    angle_3d=(25, 45),
    file="ON_finiteT_PD.pdf"
)